In [31]:
%load_ext autoreload
%autoreload 2

import os
import oddsapi
import requests
import joblib
import pandas as pd
import numpy as np
from common.utils.odds_utils import get_tournament_winner_odds, get_group_stage_odds, get_group_stage_OU
from common.utils.holdet_utils import get_holdet_data, get_play_off_opps, get_rw
from common.utils.simulation_utils import WorldCupTeam, WorldCupSim
from common.utils.group_stage import get_group_stage, playoffs
from common.utils.dict_utils import eng2dan
from scipy.optimize import fsolve
import warnings
warnings.filterwarnings('ignore')
ROUND = 1
API_KEY = os.getenv("API_KEY")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [32]:
# load data
df_holdet = get_holdet_data()
df_winner = get_tournament_winner_odds(API_KEY)
df_group_stage = get_group_stage_odds(API_KEY)
df_get_group_stage_OU = get_group_stage_OU(API_KEY)
df_groups = pd.read_csv("common/ressources/df_groups.csv")[["Team","group"]]
df_group_stage = df_group_stage[['home_team','away_team','prob1','probx', 'prob2']].merge(df_groups.rename(columns =  {"Team":"home_team"}), on = "home_team")

In [33]:
_df_group_stage = get_group_stage(df_get_group_stage_OU.copy(), df_group_stage.copy(), ROUND, eng2dan)

In [34]:
df_rw = get_rw(df_holdet)
df_rw.rename(columns = {"Player_match": "name1"}, inplace = True)
print(df_holdet.shape)
df_holdet = df_holdet.merge(df_rw, on = "name1", how ="left")
print(df_holdet.shape)
df_holdet = df_holdet[['name1', 'name2', 'country', 'price', 'position', 'Player', 'Team', 'Opp', 'Pos', 'MIN','G', 'A', 'SOG', 'Y','R']]
print(df_holdet.shape)

(830, 10)
(830, 39)
(830, 15)


In [35]:
df_holdet[df_holdet["name1"]=="Martin Braithwaite",]

,name1,name2,country,price,position,Player,Team,Opp,Pos,MIN,G,A,SOG,Y,R
419,Martin Braithwaite,M. Braithwaite,Danmark,3000000.0,Angreb,Martin Braithwaite,DEN,TUN,F,90,0.92,0.26,1.45,0.08,0.01


In [26]:
for round in [str(_) for _ in range(ROUND,4)]:
    df_temp1 = df_holdet[["name1","name2","country"]].merge(_df_group_stage.loc[_df_group_stage["round"]==round,["home_team","prob1","probx","prob2","xGH","xGA"]].rename(columns = {"home_team":"country"}), on = "country")
    df_temp1.rename(columns = {"prob1":f"prob_winning_{round}","probx":f"prob_draw_{round}","prob2":f"prob_lose_{round}","xGH":f"E_goals_{round}","xGA":f"E_goals_against_{round}"}, inplace = True)

    df_temp2 = df_holdet[["name1","name2","country"]].merge(_df_group_stage.loc[_df_group_stage["round"]==round,["away_team","prob1","probx","prob2","xGH","xGA"]].rename(columns = {"away_team":"country"}), on = "country")
    df_temp2.rename(columns = {"prob1":f"prob_lose_{round}","probx":f"prob_draw_{round}","prob2":f"prob_winning_{round}","xGA":f"E_goals_{round}","xGH":f"E_goals_against_{round}"}, inplace = True)

    df_temp_temp = pd.concat([df_temp1,df_temp2],axis=0)
    df_holdet = df_holdet.merge(df_temp_temp, on = ["name1","name2","country"])

In [27]:
df_winner  = playoffs(df_winner.copy(),_df_group_stage.copy(), eng2dan)
df_holdet = df_holdet.merge(df_winner[['country','R16_prob', 'QF_prob', 'SF_prob', 'Final_prob', 'W_prob', \
                                       'prob_to_win_R16', 'prob_to_win_QF', 'prob_to_win_SF','prob_to_win_Final', \
                                       'xG_R16', 'xG_QF', 'xG_SF', 'xG_Final','xGa_R16', 'xGa_QF', 'xGa_SF', 'xGa_Final']], on = "country")

df_holdet["selected"] = ~df_holdet['G'].isna()

In [28]:
for country in df_holdet["country"].unique():
    xG_sum = df_holdet.loc[(df_holdet["country"]==country) & (df_holdet["selected"]),'G'].astype(float).sum()
    xA_sum = df_holdet.loc[(df_holdet["country"]==country) & (df_holdet["selected"]),'A'].astype(float).sum()
    df_holdet.loc[(df_holdet["country"]==country) & (df_holdet["selected"]),'G_share'] = df_holdet.loc[(df_holdet["country"]==country) & (df_holdet["selected"]),'G'].astype(float)/xG_sum
    df_holdet.loc[(df_holdet["country"]==country) & (df_holdet["selected"]),'A_share'] = df_holdet.loc[(df_holdet["country"]==country) & (df_holdet["selected"]),'A'].astype(float)/xA_sum

    df_holdet.loc[(df_holdet["country"]==country) & (~df_holdet["selected"]),"E_points_total"] = -5000
    df_holdet.loc[(df_holdet["country"]==country) & (~df_holdet["selected"]),"E_points_round1"] = -5000
    for player in df_holdet.loc[(df_holdet["country"]==country) & (df_holdet["selected"]),"name1"]:
        points = 0
        goal_share = df_holdet.loc[df_holdet["name1"]==player,"G_share"].values[0]
        assist_share = df_holdet.loc[df_holdet["name1"]==player,"A_share"].values[0]
        yellow_prob = df_holdet.loc[df_holdet["name1"]==player,'Y'].astype(float).values[0]
        red_prob = df_holdet.loc[df_holdet["name1"]==player,'R'].astype(float).values[0]
        # group games
        loop = 4
        for i in range(ROUND,loop):
            points += 7000*(1/(i-ROUND+1))
            points += df_holdet.loc[df_holdet["name1"]==player,f'prob_winning_{i}'].values[0]*25000*(1/(i-ROUND+1))
            points += df_holdet.loc[df_holdet["name1"]==player,f'prob_draw_{i}'].values[0]*5000*(1/(i-ROUND+1))
            points += df_holdet.loc[df_holdet["name1"]==player,f'prob_lose_{i}'].values[0]*-15000*(1/(i-ROUND+1))
            points += df_holdet.loc[df_holdet["name1"]==player,f'E_goals_{i}'].values[0]*10000*(1/(i-ROUND+1))
            points += df_holdet.loc[df_holdet["name1"]==player,f'E_goals_against_{i}'].values[0]*-10000*(1/(i-ROUND+1))
            points += yellow_prob * -20000*(1/(i-ROUND+1))
            points += yellow_prob*yellow_prob*-20000*(1/(i-ROUND+1))
            points += red_prob * -50000*(1/(i-ROUND+1))
            if i == loop:
                for idx, game in enumerate(["R16","QF","SF","Final"]):
                    prob = df_holdet.loc[df_holdet["name1"]==player,f'{game}_prob'].values[0]
                    factor = 1/(2*(loop+idx))
                    points += 7000*factor
                    points += prob*np.exp(-df_holdet.loc[df_holdet["name1"]==player,f'xGa_{game}'].values[0])*-10000*factor
                    points += prob*df_holdet.loc[df_holdet["name1"]==player,f'prob_to_win_{game}'].values[0]*25000*factor
                    points += prob*(1-df_holdet.loc[df_holdet["name1"]==player,f'prob_to_win_{game}'].values[0])*-15000*factor
                    points += prob*yellow_prob * -20000*factor
                    points += prob*yellow_prob*yellow_prob*-20000*factor
                    points += prob*df_holdet.loc[df_holdet["name1"]==player,f'xG_{game}'].values[0]*10000*factor
                    points += prob*red_prob * -50000*factor
                    
            if df_holdet.loc[df_holdet["name1"]==player,"position"].values[0]=="Mål":
                points += np.exp(-df_holdet.loc[df_holdet["name1"]==player,f'E_goals_against_{i}'].values[0])*75000*(1/(i-ROUND+1))
                if i == loop:
                    for idx, game in enumerate(["R16","QF","SF","Final"]):
                        prob = df_holdet.loc[df_holdet["name1"]==player,f'{game}_prob'].values[0]
                        factor = 1/(2*(loop+idx))
                        points += prob*np.exp(-df_holdet.loc[df_holdet["name1"]==player,f'xGa_{game}'].values[0])*75000*factor
                        
            elif df_holdet.loc[df_holdet["name1"]==player,"position"].values[0]=="Forsvar":
                points += np.exp(-df_holdet.loc[df_holdet["name1"]==player,f'E_goals_against_{i}'].values[0])*50000*(1/(i-ROUND+1))
                points += goal_share*df_holdet.loc[df_holdet["name1"]==player,f'E_goals_{i}'].values[0]*175000*(1/(i-ROUND+1))
                points += assist_share*df_holdet.loc[df_holdet["name1"]==player,f'E_goals_{i}'].values[0]*60000*(1/(i-ROUND+1))
                if i == loop:
                    for idx, game in enumerate(["R16","QF","SF","Final"]):
                        prob = df_holdet.loc[df_holdet["name1"]==player,f'{game}_prob'].values[0]
                        factor = 1/(2*(loop+idx))
                        points += prob*np.exp(-df_holdet.loc[df_holdet["name1"]==player,f'xG_{game}'].values[0])*50000*factor
                        points += prob*goal_share*np.exp(-df_holdet.loc[df_holdet["name1"]==player,f'xG_{game}'].values[0])*175000*factor
                        points += prob*assist_share*np.exp(-df_holdet.loc[df_holdet["name1"]==player,f'xG_{game}'].values[0])*60000*factor
                        
            elif df_holdet.loc[df_holdet["name1"]==player,"position"].values[0]=="Midtbane":
                points += goal_share*df_holdet.loc[df_holdet["name1"]==player,f'E_goals_{i}'].values[0]*150000*(1/(i-ROUND+1))
                points += assist_share*df_holdet.loc[df_holdet["name1"]==player,f'E_goals_{i}'].values[0]*60000*(1/(i-ROUND+1))
                if i == loop:
                    for idx, game in enumerate(["R16","QF","SF","Final"]):
                        prob = df_holdet.loc[df_holdet["name1"]==player,f'{game}_prob'].values[0]
                        factor = 1/(2*(loop+idx))
                        points += prob*goal_share*np.exp(-df_holdet.loc[df_holdet["name1"]==player,f'xG_{game}'].values[0])*150000*factor
                        points += prob*assist_share*np.exp(-df_holdet.loc[df_holdet["name1"]==player,f'xG_{game}'].values[0])*60000*factor
                        
            elif df_holdet.loc[df_holdet["name1"]==player,"position"].values[0]=="Angreb":
                points += goal_share*df_holdet.loc[df_holdet["name1"]==player,f'E_goals_{i}'].values[0]*125000*(1/(i-ROUND+1))
                points += assist_share*df_holdet.loc[df_holdet["name1"]==player,f'E_goals_{i}'].values[0]*60000*(1/(i-ROUND+1))
                if i == loop:
                    for idx, game in enumerate(["R16","QF","SF","Final"]):
                        prob = df_holdet.loc[df_holdet["name1"]==player,f'{game}_prob'].values[0]
                        factor = 1/(2*(loop+idx))
                        points += prob*goal_share*np.exp(-df_holdet.loc[df_holdet["name1"]==player,f'xG_{game}'].values[0])*125000*factor
                        points += prob*assist_share*np.exp(-df_holdet.loc[df_holdet["name1"]==player,f'xG_{game}'].values[0])*60000*factor
            if i == ROUND:
                df_holdet.loc[df_holdet["name1"]==player,"E_points_round1"] = points
        df_holdet.loc[df_holdet["name1"]==player,"E_points_total"] = points

In [30]:
df_holdet.to_csv("common/ressources/df_holdet", index = False)

In [18]:
df_holdet[(df_holdet["country"]=="Danmark") & (~df_holdet["MIN"].isna())]

286    2.203827
287    2.203827
289    2.203827
290    2.203827
291    2.203827
293    2.203827
294    2.203827
300    2.203827
302    2.203827
303    2.203827
306    2.203827
Name: E_goals_1, dtype: float64

In [15]:
11*4*8

352

In [14]:
(~df_holdet["MIN"].isna()).sum()

350

In [31]:
df_rw["Team"].value_counts()

AUS    11
POR    11
IRN    11
QAT    11
MAR    11
KOR    11
SEN    11
SRB    11
CMR    11
JPN    11
SUI    11
CAN    11
DEN    11
KSA    11
CRO    11
ENG    11
ECU    11
TUN    11
CRC    11
BEL    11
ARG    11
MEX    10
WAL    10
BRA    10
URU    10
GHA    10
ESP     9
GER     9
POL     9
NED     8
USA     8
FRA     7
Name: Team, dtype: int64